# Batch EDA for CD samples

## Goals

1. Measure cell-cell distances compared to (R2 or R1?)
    - Run separately for each case
    - maybe try both R1 and R2 as reference
2. Remake the cell type abundance plots. "neighborhood_analysis" plots have outdated cell types
3. 

In [1]:
import os
import re
import anndata as ad
import numpy as np
import pandas as pd
import scanpy as sc
import scipy as sci
import seaborn as sns; sns.set(color_codes=True)
import matplotlib.pyplot as plt
%matplotlib inline 
import scimap as sm

from statsmodels.distributions.empirical_distribution import ECDF
from statsmodels.stats.multitest import multipletests
from scipy.stats import poisson
import math
import random
import tqdm
from joblib import Parallel, delayed
from multiprocessing import Pool

/home/smithd30/miniconda3/envs/scimap/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [2]:
adata = ad.read_h5ad('aSMA_CD21_analysis/all_regions_dat.h5ad')
adata.obs['case'] = adata.obs['region'].apply(lambda x: x.split("_")[0])
adata.obs['disease'] = adata.obs['case'].apply(lambda x: x[:-1])

In [3]:
def single_csr(V, p, r):
    density = p/V
    probs = 2*(density*math.pi)*(r**2)*math.exp(-density*(r**2))
    return probs
make_csr = np.vectorize(single_csr)

In [4]:
def simulate_positions(ad, c1, c2):
    dat = ad[ad.obs['new_pheno'].isin([c1,c2]),:].copy()
    ymin = np.floor(dat.obs['Absolute.Y'].min()).astype(int)
    ymax = np.floor(dat.obs['Absolute.Y'].max()).astype(int)
    xmin = np.floor(dat.obs['Absolute.X'].min()).astype(int)
    xmax = np.floor(dat.obs['Absolute.X'].max()).astype(int)
    dat.obs['Absolute.Y'] = random.choices(range(ymin, ymax), k=dat.shape[0])
    dat.obs['Absolute.X'] = random.choices(range(xmin, xmax), k=dat.shape[0])
    return dat

In [5]:
def cc_dist(ad, c1, c2):
    ref_dat = ad.obs.loc[ad.obs['new_pheno']==c1, ['Absolute.Y', 'Absolute.X']]
    q_dat = ad.obs.loc[ad.obs['new_pheno']==c2, ['Absolute.Y', 'Absolute.X']]
    ref_tree = sci.spatial.cKDTree(ref_dat)
    dist_res = ref_tree.query(q_dat, k=1)
    ecdf = ECDF(dist_res[0])
    return(ecdf)

In [6]:
# making the CSR
def simdata(adata, ref_dat, q_dat, ecdf):
    y = np.ptp(adata.obs['Absolute.Y'])
    x = np.ptp(adata.obs['Absolute.X'])
    area = y*x
    events = ref_dat.shape[0] + q_dat.shape[0]
    dist = np.arange(1,np.floor(ecdf.x.max()), 5)
    csr_prob = make_csr(area, events, dist)
    csr_prob = csr_prob/csr_prob.sum()
    csr_cdf = np.array(list(map(lambda x: csr_prob[0:x].sum(), np.arange(len(dist)))))
    return (dist, csr_cdf)

In [7]:
# cell-cell distances
def runCCtest(adata, ref_name, case_name):
    if ref_name == case_name:
        return None
    case_dat = adata[(adata.obs.case == case_name) | (adata.obs.case == ref_name),:].copy()
    itxn_dir = os.path.join('outs',case_name[:-1],'cell-cell_itxn_' + ref_name +'ref', case_name)
    if not os.path.isdir(itxn_dir):
            os.mkdir(itxn_dir)
            
    cc_itxn_dict = {}
    for c in tqdm.tqdm(case_dat.obs['new_pheno'].unique()):
        if not os.path.isdir(os.path.join(itxn_dir,c)):
            os.mkdir(os.path.join(itxn_dir,c))
        for cc in case_dat.obs['new_pheno'].unique():
            if c == cc:
                continue
            if np.sum(case_dat.obs.loc[case_dat.obs.case == ref_name, 'new_pheno']==c) < 100 or np.sum(case_dat.obs.loc[case_dat.obs.case==ref_name, 'new_pheno']==cc) < 100: 
                continue
            # if os.path.isfile(os.path.join(itxn_dir, c, c + '-' + cc + '_itxn.png')): # in case of a failed run
            #     continue
            c_key = c + ' -- ' + cc
            cc_itxn_dict[c_key] = []
            dis_dict = {}
            dist = None
            for cname in case_dat.obs.case.unique():
                dat_dict = []
                for r in case_dat.obs.loc[case_dat.obs['case']==cname, 'region'].unique():
                    adata_sub = case_dat[case_dat.obs.region == r,:].copy()
                    ref_dat = adata_sub.obs.loc[adata_sub.obs['new_pheno']==c, ['Absolute.Y', 'Absolute.X']]
                    q_dat = adata_sub.obs.loc[adata_sub.obs['new_pheno']==cc, ['Absolute.Y', 'Absolute.X']]
                    ref_tree = sci.spatial.cKDTree(ref_dat)
                    dist_res = ref_tree.query(q_dat, k=1)
                    ecdf = ECDF(dist_res[0])
                    if dist is None: # just going to take the first region we get
                        dist = np.arange(1,np.floor(ecdf.x.max()), 1) # what should the distance be? dist = np.arange(1,np.floor(ecdf.x.max()), 5) of R?
                        dis_dict['dist'] = pd.Series(dist)
                    dat_dict.append(pd.Series(ecdf(dist)))
                merge_df = pd.concat(dat_dict, axis=1)
                # for r, df in dat_dict.items():
                #     if merge_df is None:
                #         merge_df = dat_dict[r]
                #     else:
                #         merge_df = merge_df.join(df)
                dis_dict[cname] = merge_df.apply(np.mean, axis=1)
            dis_merge_df = pd.DataFrame(dis_dict)
            cc_itxn_dict[c_key].append(dis_merge_df['dist'][np.argmin(np.absolute(dis_merge_df[ref_name]-0.5))])
            cc_itxn_dict[c_key].append(dis_merge_df['dist'][np.argmin(np.absolute(dis_merge_df[case_name]-0.5))])
            stat = sci.stats.ks_2samp(dis_merge_df[ref_name], dis_merge_df[case_name])
            cc_itxn_dict[c_key].append(stat[1])

            rplot, = plt.plot(dis_merge_df['dist'], dis_merge_df[ref_name])
            kfdplot, = plt.plot(dis_merge_df['dist'], dis_merge_df[case_name], c='red')
            plt.legend([rplot, kfdplot], [ref_name, case_name], loc = 'lower right')
            plt.savefig(os.path.join(itxn_dir, c, c + '-' + cc + '_itxn.png'))
            plt.close()
    cc_df = pd.DataFrame(cc_itxn_dict).T
    cc_df.columns = ['avg_'+ref_name+'_distance', 'avg_'+case_name+'_distance', 'p_val']
    cc_df['p_val_adj'] = multipletests(cc_df['p_val'], method='bonferroni')[1]
    cc_df.to_csv(os.path.join(itxn_dir, "stats.csv"))
    return None


In [ ]:
runCCtest(adata, 'R2', 'MCD3')

  5%|▌         | 1/19 [00:40<12:13, 40.72s/it]

In [9]:
# Running it!
ref = 'R1'
res = Parallel(4)(delayed(runCCtest)(adata, ref, case_name) for case_name in adata.obs['case'].unique())

100%|██████████| 19/19 [40:18<00:00, 127.31s/it]
/mnt/isilon/cscb/software/anaconda3/envs/scimap/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.

100%|██████████| 19/19 [41:36<00:00, 131.41s/it]
